In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

In [4]:
df = pd.read_parquet("DSCB310 - UE2 - Shopping Carts.parquet")

In [5]:
df.reset_index(drop= True)

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county
0,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo
1,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo
2,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo
3,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo
4,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132664,606914,28849,6,1,193225,18,5,13,4.0,1,No Salt Added Black Beans,59,15,canned goods,canned meals beans,Calaveras
6132665,606915,27966,1,0,107102,1,3,10,NaN,0,Organic Raspberries,123,4,produce,packaged vegetables fruits,Amador
6132666,606915,27845,4,0,107102,1,3,10,NaN,0,Organic Whole Milk,84,16,dairy eggs,milk,Amador
6132667,606915,26940,3,0,107102,1,3,10,NaN,0,Organic Large Green Asparagus,83,4,produce,fresh vegetables,Amador


### **P3:** Welche Attribute einer Bestellung wirken sich auf das Trinkgeldverhalten aus?

In [ ]:
# feature engineering von p_ziele mit einbringen
# add_to_card_order anschauen

# correlation, user_id, add_to_cart_order, product_name, order_dow, order_size machen !

#### Correlation

In [ ]:
# nach order_id gruppieren und dann corr()

In [6]:
df.head()

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo


In [11]:
df_corr = df.groupby("order_id").agg(
 product_it = ("product_id", "unique"), 
 order_hour_of_day= ("order_hour_of_day", "mean"), 
 days_since_prior_order = ("days_since_prior_order", "mean"), 
 tip = ("tip", "mean"), 
 department_id = ("department_id", "unique"), 
 county= ("county", "max"),
 aisle_id = ("aisle_id", "unique"),
 user_id = ("user_id", "unique")
 ).reset_index()

In [12]:
df_corr.corr()

,order_id,order_hour_of_day,days_since_prior_order,tip
order_id,1.000000,0.000546,0.000495,-0.001533
order_hour_of_day,0.000546,1.000000,0.000152,0.024662
days_since_prior_order,0.000495,0.000152,1.000000,-0.133145
tip,-0.001533,0.024662,-0.133145,1.000000


In [ ]:
df[["product_id", "reordered", "order_dow", "order_hour_of_day", "days_since_prior_order", "tip"]].corr()

,product_id,reordered,order_dow,order_hour_of_day,days_since_prior_order,tip
product_id,1.000000,0.003051,-0.002709,0.001654,0.000652,-0.001380
reordered,0.003051,1.000000,-0.011324,-0.022744,-0.130482,0.077467
order_dow,-0.002709,-0.011324,1.000000,0.009725,-0.030760,-0.114064
order_hour_of_day,0.001654,-0.022744,0.009725,1.000000,0.004329,0.025336
days_since_prior_order,0.000652,-0.130482,-0.030760,0.004329,1.000000,-0.143219
tip,-0.001380,0.077467,-0.114064,0.025336,-0.143219,1.000000


In [ ]:
df.loc[df.tip == 1].head()

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo


#### Order_dow

In [ ]:
df1 = df.groupby(["order_dow", "order_id"], as_index= False).agg(tip_orders_per_day = ("tip", "mean"))
df1

,order_dow,order_id,tip_orders_per_day
0,0,4,1.0
1,0,6,0.0
2,0,9,1.0
3,0,10,0.0
4,0,19,1.0
...,...,...,...
606910,6,606886,1.0
606911,6,606898,1.0
606912,6,606904,0.0
606913,6,606906,1.0


In [ ]:
df_order_dow= df1.groupby("order_dow", as_index= False).agg(tip = ("tip_orders_per_day","sum"), order_id= ("order_id", "nunique"))
df_order_dow["tip_norm"] = df_order_dow.tip / df_order_dow.order_id
df_order_dow

,order_dow,tip,order_id,tip_norm
0,0,53749.0,106842,0.503070
1,1,54928.0,104713,0.524558
2,2,30614.0,83177,0.368058
3,3,27884.0,77329,0.360589
4,4,27780.0,75323,0.368812
5,5,30188.0,80360,0.375660
6,6,28234.0,79171,0.356620


**Tippwahrscheinlichkeit für jeden Wochentag**

In [ ]:
px.bar(df_order_dow, x= "order_dow", y= "tip_norm")

#### Order_hour_of_day

In [6]:
df_h = df.groupby(["order_hour_of_day", "order_id"], as_index= False).agg(tip= ("tip", "mean"))
df_hour = df_h.groupby(["order_hour_of_day"], as_index= False).agg(orders= ("order_id", "nunique"), tip= ("tip", "sum"))
df_hour["tip_norm"] = df_hour.tip / df_hour.orders
df_hour["orders_norm"] = df_hour.orders / df_hour.orders.sum()
df_hour

,order_hour_of_day,orders,tip,tip_norm,orders_norm
0,0,4072,2079.0,0.510560,0.006709
1,1,2219,1125.0,0.506985,0.003656
2,2,1340,694.0,0.517910,0.002208
3,3,956,477.0,0.498954,0.001575
4,4,986,510.0,0.517241,0.001625
5,5,1644,619.0,0.376521,0.002709
6,6,5461,2146.0,0.392968,0.008998
7,7,16293,6641.0,0.407598,0.026846
8,8,31847,13411.0,0.421107,0.052474
9,9,45728,19502.0,0.426478,0.075345


**Tippwahrscheinlichkeit pro Stunde**

In [107]:
fig1 = make_subplots(cols= 1, rows= 2, shared_xaxes= True, shared_yaxes= False, vertical_spacing= 0)

for order_hour_of_day in df_hour.order_hour_of_day:

 if df_hour[df_hour.order_hour_of_day == order_hour_of_day].tip_norm.values >= 0.5:
  color = px.colors.qualitative.G10[4]
  text= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm.values.round(3)

  fig1.add_trace(
   go.Bar(
    x= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day,
    y= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm,
    name= f"{order_hour_of_day}: {(df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{order_hour_of_day}" 
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day,
    y= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm,
    name= f"Orders: {(df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm.values[0] * 100).round(3)}%",
    base = -df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{order_hour_of_day}"
   ),
   col= 1,
   row= 2
  )

 elif 0.4 < df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm.values < 0.5:
  color= px.colors.sequential.Greys[3]

  fig1.add_trace(
   go.Bar(
    x= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day,
    y= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm,
    name= f"{order_hour_of_day}: {(df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{order_hour_of_day}"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day,
    y= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm,
    name= f"Orders: {(df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm.values[0] * 100).round(3)}%",
    base = -df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{order_hour_of_day}"
   ),
   col= 1,
   row= 2
  )

 elif 0.3 < df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm.values < 0.4:
  color= px.colors.sequential.Greys[2]

  fig1.add_trace(
   go.Bar(
    x= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day,
    y= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm,
    name= f"{order_hour_of_day}: {(df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{order_hour_of_day}"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].order_hour_of_day,
    y= df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm,
    name= f"Orders: {(df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm.values[0] * 100).round(3)}%",
    base = -df_hour.loc[df_hour.order_hour_of_day == order_hour_of_day].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{order_hour_of_day}"
   ),
   col= 1,
   row= 2
  )

fig1.add_hline(
 y= 0,
 opacity= 0.3,
 line_dash= "solid",
 line_color= "grey",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= 0,
 opacity= 0.3,
 line_dash= "solid",
 line_color= "grey",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= -0.1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 2
)

fig1.update_annotations(opacity= 0.8)
fig1.update_layout(title= dict(text= "<b>Tippwahrscheinlichkeit nach Tagesstunden</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), legend_title_text = "<b>Stunden</b>", height= 500)
fig1.update_xaxes(title_text="<b>Stunden</b>", ticks= "outside", col= 1, row= 2)
fig1.update_xaxes(showline= False, col= 1, row= 1)
fig1.update_yaxes(title_text="<b>Tipp Wshk.</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 1, showline= False)
fig1.update_yaxes(title_text="<b>Order Anteil</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 2, tickvals= [0, -0.05, -0.075, -0.1])
fig1.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))
fig1.show()

#### user_id

In [ ]:
df_user = df.groupby(["user_id", "order_id"], as_index= False).agg(user_tip_mean= ("tip", "mean"))
df_user_tip = df_user.groupby("user_id", as_index= False).agg(tip= ("user_tip_mean", "sum"), orders = ("order_id", "nunique"))
df_user_tip["tip_norm"] = df_user_tip.tip / df_user_tip.orders
df_user_tip

,user_id,tip,orders,tip_norm
0,3,10.0,12,0.833333
1,7,12.0,21,0.571429
2,14,8.0,14,0.571429
3,23,0.0,5,0.000000
4,25,2.0,3,0.666667
...,...,...,...,...
37362,206186,2.0,4,0.500000
37363,206189,2.0,6,0.333333
37364,206192,2.0,14,0.142857
37365,206195,3.0,20,0.150000


In [ ]:
# pd.cut machen und dann als bar plot 

In [ ]:
#feature engineering aus p_ziel und mit p4 argumentieren

#### product_name

In [ ]:
df_product_tip = df.groupby("product_name", as_index= False).agg(tip_per_product= ("tip", "sum"), orders= ("index", "count")).sort_values(by= "tip_per_product", ascending= False)

In [ ]:
df_product_tip["tip_rate_per_product"] = df_product_tip.tip_per_product / df_product_tip.orders
df_product_tip.loc[df_product_tip.orders >= 10000]

,product_name,tip_per_product,orders,tip_rate_per_product
3369,Banana,48084,89127,0.539500
3177,Bag of Organic Bananas,37468,70283,0.533102
29385,Organic Strawberries,23033,49961,0.461020
26443,Organic Baby Spinach,19480,45462,0.428490
27836,Organic Hass Avocado,18013,39290,0.458463
26408,Organic Avocado,14419,32896,0.438321
20554,Large Lemon,12205,28495,0.428321
29920,Organic Whole Milk,12078,25309,0.477222
28851,Organic Raspberries,11907,25589,0.465317
21486,Limes,11608,26443,0.438982


In [1]:
# ploten als bar oder pie

#### Order_size

In [ ]:
df_order_size = df.groupby("order_id", as_index= False).agg(order_size= ("order_id", "count"), tip= ("tip", "sum"))
df_order_size["tip"] = df_order_size.tip / df_order_size.order_size
df_order_size

,order_id,order_size,tip
0,1,8,1.0
1,2,3,0.0
2,3,5,0.0
3,4,3,1.0
4,5,2,0.0
...,...,...,...
606910,606911,6,0.0
606911,606912,13,1.0
606912,606913,19,0.0
606913,606914,8,1.0


In [ ]:
df_order_size.loc[df_order_size.tip == 1].order_size.mean()

10.831081747751375

In [ ]:
df_order_size.loc[df_order_size.tip == 0].order_size.mean()

9.584039056621918

In [ ]:
df.groupby(["user_id", "order_id", "order_number"]).agg(count= ("order_number", "count"))

count
user_id order_id order_number       
3       78693    2                 9
        92370    11                5
        119980   10                6
        244029   1                10
        248921   12                6
...                              ...
206201  488887   13               16
        550775   32               12
        554910   4                14
        575966   16                8
        595676   22                4

[606915 rows x 1 columns]

In [ ]:
df_order_ = df.groupby("order_id", as_index= False).agg(order_size= ("order_number", "max"), tip= ("tip", "max"))
df_order_.loc[df_order_.tip == 1].order_size.mean()

18.91474758956022

In [ ]:
df_order_.loc[df_order_.tip == 0].order_size.mean()

15.851800937947264

#### days_since_prior_order

In [55]:
df_days = df.groupby(["days_since_prior_order", "order_id"], as_index=False).agg(tip_mean= ("tip", "mean"))
df_days_since_prior_order= df_days.groupby("days_since_prior_order", as_index= False).agg(tip= ("tip_mean", "sum"), orders= ("order_id", "nunique"))
df_days_since_prior_order["tip_norm"] = df_days_since_prior_order.tip / df_days_since_prior_order.orders
df_days_since_prior_order["orders_norm"] = df_days_since_prior_order.orders / df_days_since_prior_order.orders.sum()
df_days_since_prior_order

,days_since_prior_order,tip,orders,tip_norm,orders_norm
0,0.0,5159.0,12005,0.429738,0.021078
1,1.0,10188.0,26178,0.389182,0.045963
2,2.0,14490.0,34983,0.414201,0.061422
3,3.0,17048.0,38541,0.442334,0.067669
4,4.0,18052.0,39479,0.457256,0.069316
5,5.0,17911.0,38254,0.468212,0.067166
6,6.0,21052.0,42605,0.494120,0.074805
7,7.0,31308.0,57068,0.548609,0.100199
8,8.0,15924.0,32510,0.489819,0.057080
9,9.0,9502.0,21219,0.447806,0.037256


In [99]:
fig1 = make_subplots(cols= 1, rows= 2, shared_xaxes= True, shared_yaxes= False, vertical_spacing= 0)

for days_since_prior_order in df_days_since_prior_order.days_since_prior_order:

 if df_days_since_prior_order[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values >= 0.5:
  color = px.colors.qualitative.G10[4]
  text= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values.round(2)

  fig1.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm,
    name= f"{int(days_since_prior_order)}: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{days_since_prior_order}" 
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm,
    name= f"Orders: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm.values[0] * 100).round(3)}%",
    base = -df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{days_since_prior_order}"
   ),
   col= 1,
   row= 2
  )

 elif 0.4 < df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values < 0.5:
  color= px.colors.sequential.Greys[3]

  fig1.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm,
    name= f"{int(days_since_prior_order)}: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{days_since_prior_order}"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm,
    name= f"Orders: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm.values[0] * 100).round(3)}%",
    base = -df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{days_since_prior_order}"
   ),
   col= 1,
   row= 2
  )

 elif 0.3 < df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values < 0.4:
  color= px.colors.sequential.Greys[2]

  fig1.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm,
    name= f"{int(days_since_prior_order)}: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{days_since_prior_order}"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm,
    name= f"Orders: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm.values[0] * 100).round(3)}%",
    base = -df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{days_since_prior_order}"
   ),
   col= 1,
   row= 2
  )

 elif 0.2 < df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values < 0.3:
  color= px.colors.sequential.Greys[1]
  text= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values.round(2)

  fig1.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm,
    name= f"{int(days_since_prior_order)}: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{days_since_prior_order}"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].days_since_prior_order,
    y= df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm,
    name= f"Orders: {(df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm.values[0] * 100).round(3)}%",
    base = -df_days_since_prior_order.loc[df_days_since_prior_order.days_since_prior_order == days_since_prior_order].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{days_since_prior_order}"
   ),
   col= 1,
   row= 2
  )


fig1.add_hline(
 y= 0,
 opacity= 0.3,
 line_dash= "solid",
 line_color= "grey",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= 0,
 opacity= 0.3,
 line_dash= "solid",
 line_color= "grey",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= -0.18,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 2
)

fig1.update_annotations(opacity= 0.8)
fig1.update_layout(title= dict(text= "<b>Tippwahrscheinlichkeit von Bestellungen mit Tagen seit der letzten Bestellungen</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), legend_title_text = "<b>Tage</b>", height= 700)
fig1.update_xaxes(title_text="<b>Tage</b>", ticks= "outside", col= 1, row= 2)
fig1.update_xaxes(showline= False, col= 1, row= 1)
fig1.update_yaxes(title_text="<b>Tipp Wshk.</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 1, showline= False)
fig1.update_yaxes(title_text="<b>Order Anteil</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 2)
fig1.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))
fig1.show()

#### department

In [46]:
df_dep = df.groupby(["department", "order_id"], as_index= False).agg(tip_mean= ("tip", "mean"), all= ("order_id", "count"))
df_dep1 = df_dep.groupby("department", as_index= False).agg(tip = ("tip_mean", "sum"), orders= ("order_id", "nunique"))
df_dep1["tip_norm"] = df_dep1.tip / df_dep1.orders
df_dep1["orders_norm"] = df_dep1.orders / df_dep1.orders.sum()
df_dep2 = df_dep1.sort_values(by= "tip_norm", ascending= False)

In [47]:
df_dep2

,department,tip,orders,tip_norm,orders_norm
0,alcohol,12531.0,15569,0.804869,0.005399
4,breakfast,53783.0,100391,0.535735,0.034813
1,babies,17786.0,33891,0.524800,0.011753
5,bulk,3448.0,6618,0.521003,0.002295
20,snacks,133047.0,263766,0.504413,0.091468
8,deli,68264.0,146512,0.465928,0.050807
16,pantry,97742.0,211817,0.461445,0.073453
12,international,19089.0,41853,0.456096,0.014514
7,dairy eggs,182203.0,410949,0.443371,0.142508
15,other,2914.0,6759,0.431129,0.002344


In [97]:
fig1 = make_subplots(cols= 1, rows= 2, shared_xaxes= True, shared_yaxes= False, vertical_spacing= 0)

for department in df_dep2.department:

 if df_dep2[df_dep2.department == department].tip_norm.values >= 0.5:
  color = px.colors.qualitative.G10[4]
  text= df_dep2.loc[df_dep2.department == department].tip_norm.values.round(3)

  fig1.add_trace(
   go.Bar(
    x= df_dep2.loc[df_dep2.department == department].department,
    y= df_dep2.loc[df_dep2.department == department].tip_norm,
    name= f"{department}: {(df_dep2.loc[df_dep2.department == department].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{department}" 
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_dep2.loc[df_dep2.department == department].department,
    y= df_dep2.loc[df_dep2.department == department].orders_norm,
    name= f"Orders: {(df_dep2.loc[df_dep2.department == department].orders_norm.values[0] * 100).round(3)}%",
    base = -df_dep2.loc[df_dep2.department == department].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{department}"
   ),
   col= 1,
   row= 2
  )

 elif 0.4 < df_dep2.loc[df_dep2.department == department].tip_norm.values < 0.5:
  color= px.colors.sequential.Greys[3]

  fig1.add_trace(
   go.Bar(
    x= df_dep2.loc[df_dep2.department == department].department,
    y= df_dep2.loc[df_dep2.department == department].tip_norm,
    name= f"{department}: {(df_dep2.loc[df_dep2.department == department].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{department}"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_dep2.loc[df_dep2.department == department].department,
    y= df_dep2.loc[df_dep2.department == department].orders_norm,
    name= f"Orders: {(df_dep2.loc[df_dep2.department == department].orders_norm.values[0] * 100).round(3)}%",
    base = -df_dep2.loc[df_dep2.department == department].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{department}"
   ),
   col= 1,
   row= 2
  )

 elif 0.3 < df_dep2.loc[df_dep2.department == department].tip_norm.values < 0.4:
  color= px.colors.sequential.Greys[2]

  fig1.add_trace(
   go.Bar(
    x= df_dep2.loc[df_dep2.department == department].department,
    y= df_dep2.loc[df_dep2.department == department].tip_norm,
    name= f"{department}: {(df_dep2.loc[df_dep2.department == department].tip_norm.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{department}"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_dep2.loc[df_dep2.department == department].department,
    y= df_dep2.loc[df_dep2.department == department].orders_norm,
    name= f"Orders: {(df_dep2.loc[df_dep2.department == department].orders_norm.values[0] * 100).round(3)}%",
    base = -df_dep2.loc[df_dep2.department == department].orders_norm,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{department}"
   ),
   col= 1,
   row= 2
  )

fig1.add_hline(
 y= 0,
 opacity= 0.3,
 line_dash= "solid",
 line_color= "grey",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= 0,
 opacity= 0.3,
 line_dash= "solid",
 line_color= "grey",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= -0.18,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 col= 1,
 row= 2
)

fig1.update_annotations(opacity= 0.8)
fig1.update_layout(title= dict(text= "<b>Tippwahrscheinlichkeit der Departments</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), legend_title_text = "<b>Counties</b>", height= 700)
fig1.update_xaxes(title_text="<b>Counties</b>", ticks= "outside", col= 1, row= 2)
fig1.update_xaxes(showline= False, col= 1, row= 1)
fig1.update_yaxes(title_text="<b>Tipp Wshk.</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 1, showline= False)
fig1.update_yaxes(title_text="<b>Order Anteil</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 2)
fig1.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))
fig1.show()